In [29]:
from target import avg_data_2013, avg_data_2014,avg_data_2015,avg_data_2016,avg_data_2017 ,get_html
from bs4 import BeautifulSoup as bs
import os
import sys
import csv
import pandas as pd

In [2]:
def independent_data(month,year):
    data_html=open(f"data/html_data/{year}/{month}.html","r")
    data=data_html.read()
    no_of_columns=15
    final_data=[]
    
    
    temp_data=[]
    soup=bs(data)
    table=soup.find("table",{"class":"medias mensuales numspan"})
    for tbody in table:
        for table_row in tbody:
            temp_data.append(table_row.get_text())
            
    #calculate the number of rows
    no_of_rows=len(temp_data)/no_of_columns
    
    for _ in range(round(no_of_rows)):
        new_temp_data=[]
        for i in range(15):
            new_temp_data.append(temp_data[0])
            temp_data.pop(0)
        final_data.append(new_temp_data)
    #remove unwanted rows
    final_data.pop(-1)#remove the last rows
    final_data.pop(0)#remove the 1st rows
    
    #remove unwanted rows
    for i in range(len(final_data)):
        final_data[i].pop(6)
        final_data[i].pop(13)
        final_data[i].pop(12)
        final_data[i].pop(11)
        final_data[i].pop(10)
        final_data[i].pop(9)
        final_data[i].pop(0)
        
    return final_data

In [23]:
get_html(2013,2017)

In [26]:
#independent_data_target-data..combine
if not os.path.exists("data/real_data"):
    os.makedirs("data/real_data")

for year in range(2013,2017):
    final_data = []
    with open(f"data/real_data/real_{year}.csv","w") as csvfile:
        wr=csv.writer(csvfile,dialect="excel")
        wr.writerow(["T","TM","tm","SLP","H","VV","V","VM","PM 2.5"])
        
    for month in range(1,13):
        temp_data=independent_data(month,year)
        final_data+=temp_data
        
    pm = getattr(sys.modules[__name__],f"avg_data_{year}")()
    for i in range(len(final_data)-1):
        final_data[i].insert(8,pm[i])
    
    with open(f"data/real_data/real_{year}.csv","a") as csvfile:
        wr=csv.writer(csvfile,dialect="excel")
        for row in final_data:
            flag=0
            for elem in row:
                if elem in ["","-"]:
                    flag=1
            if flag != 1:
                wr.writerow(row)

In [32]:
def year_range(start,end):
    df_list=[]
    for year in range(start,end):
        df=pd.read_csv(f"data/real_data/real_{year}.csv")
        df_list.append(df)
    return pd.concat(df_list,ignore_index=True)

In [34]:
year_range(2013,2017)

,T,TM,tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61,1.3,8.7,22.2,200.645833
...,...,...,...,...,...,...,...,...,...
726,18.1,24.0,11.2,1015.4,56,1.8,15.9,25.9,288.416667
727,17.8,25.0,10.7,1015.8,54,2.3,9.4,22.2,256.833333
728,13.9,24.5,11.4,1015.0,95,0.6,8.7,14.8,169.000000
729,16.3,23.0,9.8,1016.9,78,1.1,7.4,16.5,186.041667


In [ ]:
year_range(2013,2017).to